In [ ]:
import pandas as pd
import numpy as np
from os.path import join
import math

from google.colab import drive

# authentication for google sheets
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
write_directory = '/content/drive/My Drive/covid19data/'

In [ ]:
# region csv is located at: https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip

#download the zip file
!wget https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip

#unzip the file
!unzip Region_Mobility_Report_CSVs.zip

#get the AU file
REGION = "AU"
df2020 = pd.read_csv("2020_"+REGION+"_Region_Mobility_Report.csv")
df2021 = pd.read_csv("2021_"+REGION+"_Region_Mobility_Report.csv")
df = df2020.append(df2021)

df.head()

--2021-07-25 11:17:54--  https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip
Resolving www.gstatic.com (www.gstatic.com)... 173.194.215.94, 2607:f8b0:400c:c09::5e
Connecting to www.gstatic.com (www.gstatic.com)|173.194.215.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52808975 (50M) [application/zip]
Saving to: ‘Region_Mobility_Report_CSVs.zip.1’

Region_Mobility_Rep 100%[===================>]  50.36M  70.9MB/s    in 0.7s    

2021-07-25 11:17:54 (70.9 MB/s) - ‘Region_Mobility_Report_CSVs.zip.1’ saved [52808975/52808975]

Archive:  Region_Mobility_Report_CSVs.zip
replace 2020_AE_Region_Mobility_Report.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#inspect the data (see first 5 rows)
df.columns

In [ ]:
#notes for AU:
#df.sub_region_1 = State = [, 'Australian Capital Territory', 'New South Wales', 'Northern Territory', 'Queensland', 'South Australia', 'Tasmania', 'Victoria', 'Western Australia']
#df.sub_region_2 = councils
#df.metro_area = null
#df.iso_3166_2_code.unique() = [nan, 'AU-ACT', 'AU-NSW', 'AU-NT', 'AU-QLD', 'AU-SA', 'AU-TAS','AU-VIC', 'AU-WA']
#df[df.iso_3166_2_code=='AU-NSW'].date.tail()
#df[df.date==max(df.date)]

## Get place_id lookup (equivalent of index_v)
Google assigned a unique `place_id` for each Country / sub_region_1 / sub_region_2...etc

For each unique place_id, we will get 3 fields:
- Country / sub_region_1 / sub_region_2

In [ ]:
# initialise an empty dataframe for the place_id lookup
place_id_lookup = pd.DataFrame()

for placeid in df.place_id.unique():
  #ensure there is only one country_region for this placeid
  if (len(df[df['place_id']==placeid]['country_region'].unique())!=1):
    print("ERROR!!!")
  country_region = df[df['place_id']==placeid]['country_region'].unique()[0]

  #ensure there is only one iso_3166_2_code for this placeid
  if (len(df[df['place_id']==placeid]['iso_3166_2_code'].unique())!=1):
    print("ERROR!!!")
  iso_3166_2_code = df[df['place_id']==placeid]['iso_3166_2_code'].unique()[0]

  #ensure there is only one sub_region_1 for this placeid
  if (len(df[df['place_id']==placeid]['sub_region_1'].unique())!=1):
    print("ERROR!!!")
  sub_region_1 = df[df['place_id']==placeid]['sub_region_1'].unique()[0]

  #ensure there is only one sub_region_2 for this placeid
  if (len(df[df['place_id']==placeid]['sub_region_2'].unique())!=1):
    print("ERROR!!!")
  sub_region_2 = df[df['place_id']==placeid]['sub_region_2'].unique()[0]


  new_row = {'place_id'       : placeid,
             'iso_3166_2_code': iso_3166_2_code,
             'country_region' : country_region, 
             'sub_region_1'   : sub_region_1, 
             'sub_region_2'   : sub_region_2
            }

  #append row to the dataframe
  place_id_lookup = place_id_lookup.append(new_row, ignore_index=True)

#inspect the place_id_lookup
place_id_lookup

In [ ]:
def get_timestamp():
  from datetime import datetime, timezone

  #get sydney timezone
  import pytz
  sydney_tz = pytz.timezone('Australia/Sydney')
  now = datetime.now(sydney_tz)

  # format to dd/mm/YY H:M:S with timezone
  dt_string = now.strftime("%d/%m/%Y %H:%M:%S %z")
  return dt_string

get_timestamp()

In [ ]:
def write_to_gsheet(workbook_title, worksheet_title, data):
  # open spreadsheet
  sh = gc.open(workbook_title)

  #select an existing worksheet (tab)
  worksheet = sh.worksheet(worksheet_title)

  #delete that worksheet
  sh.del_worksheet(worksheet)

  #recreate worksheet (tab) and populate with dataframe
  worksheet = sh.add_worksheet(title=worksheet_title, rows="1000", cols="200")
  set_with_dataframe(worksheet, data, include_index=True)

1. Pivot the data
2. Move the 'Australia' label down to the other levels
3. Move the State labels down to the sub_region_2



 

In [ ]:
CATEGORIES = ['retail_and_recreation_percent_change_from_baseline',
              'grocery_and_pharmacy_percent_change_from_baseline',
              'parks_percent_change_from_baseline',
              'transit_stations_percent_change_from_baseline',
              'workplaces_percent_change_from_baseline',
              'residential_percent_change_from_baseline'
            ]

# update the fields for AU (sub regions are null for Country)
# want unique state values excluding null
df.loc[(df.sub_region_1.isnull()) & (df.sub_region_2.isnull()) & (df.iso_3166_2_code.isnull()) & (df['country_region']=='Australia'), 'sub_region_1'] = 'AUS'
df.loc[(df.sub_region_2.isnull()) & (df.iso_3166_2_code.isnull()) & (df['country_region']=='Australia'), 'sub_region_2'] = 'AUS'

# move the iso_3166_2_code (i.e. state) into sub_region_2
# want unique state values excluding null
for state in df.iso_3166_2_code.copy().dropna().unique():
  df.loc[df['iso_3166_2_code']==state, 'sub_region_2'] = state[3:]


cols = ['sub_region_1', 'sub_region_2']


# for each category, pivot on that category
for category in CATEGORIES:
  print("processing {}".format(category))
  mobility_data = df.pivot(index = 'date', columns = cols, values=category)

  #get rolling average
  rolling_mean_df = mobility_data.rolling(7).mean()

  #add timestamp
  mobility_data['last_updated'] = get_timestamp()
  mobility_data_pivoted['last_updated'] = get_timestamp()
  rolling_mean_df['last_updated'] = get_timestamp()

  #write to googlesheets
  write_to_gsheet(workbook_title='mobility_data',
                worksheet_title = category.split('_')[0],
                data = mobility_data)
  
  write_to_gsheet(workbook_title='mobility_data_rolling_7day_average',
                worksheet_title = category.split('_')[0],
                data = rolling_mean_df)

In [ ]:
print("SUCCESS - script has run as of {}".format(get_timestamp()))

In [ ]:
#get LGA data just before lockdown
test = df[df.date>='2021-06-01']
test = test[['date', 'sub_region_1', 'sub_region_2']+CATEGORIES]

write_to_gsheet(workbook_title='TEST_mobility_data_pivoted',
              worksheet_title = 'all_categories',
              data = test)